In [ ]:
# %pip installs (Studio/Studio Lab)
%pip -q install sagemaker==2.219.0 boto3 scikit-learn==1.5.1 pandas

import sagemaker, boto3, os, json, pandas as pd
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role

# Session & role
session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()  # in Studio/Studio Lab w/ IAM passthrough
prefix = "wine-quality-no-container"

# Data prep
UCI_RED = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
UCI_WHITE = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
red = pd.read_csv(UCI_RED, sep=";")
white = pd.read_csv(UCI_WHITE, sep=";")
df = pd.concat([red, white], ignore_index=True)
df.to_csv("train.csv", index=False)

# Upload to S3
train_s3 = session.upload_data("train.csv", bucket=bucket, key_prefix=f"{prefix}/input/train")
train_s3



In [ ]:
# Estimator using SageMaker-managed SKLearn container
estimator = SKLearn(
    entry_point="train.py",
    source_dir="../src",        # <- points to sagemaker/src/train.py
    role=role,
    instance_type="ml.m5.large",
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=session,
    hyperparameters={"target-col": "quality"},
)

# Train
estimator.fit({"train": train_s3})


In [ ]:
# Deploy with custom inference handlers
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    entry_point="inference.py",
    source_dir="../src",
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)


In [ ]:
# Invoke
sample = {
 "fixed acidity": 7.4, "volatile acidity": 0.70, "citric acid": 0.00,
 "residual sugar": 1.9, "chlorides": 0.076, "free sulfur dioxide": 11.0,
 "total sulfur dioxide": 34.0, "density": 0.9978, "pH": 3.51,
 "sulfates": 0.56, "alcohol": 9.4
}
predictor.predict({"features": sample})


In [ ]:
# Cleanup
predictor.delete_endpoint()
